In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client=MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
client.search_experiments()

[<Experiment: artifact_location=("file:c:/Sutiwas's Documents/GitHub "
  'Projects/mlops-journey/02-experiment-tracking/mlruns/1'), creation_time=1747212136301, experiment_id='1', last_update_time=1747212136301, lifecycle_stage='active', name='TrialExperimentNYC_Taxi', tags={}>]

In [7]:
client.create_experiment(name="my-cool-experiment")

'2'

In [27]:
from mlflow.entities import Experiment, ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [28]:
for run in runs:
    try:
        print(f"run_id: {run.info.run_id}, run_name: {run.info.run_name}, rmse: {run.data.metrics['val_rmse']:.4f}")
    except KeyError:
        print(f"run_id: {run.info.run_id}, run_name: {run.info.run_name}, val_rmse: {run.data.metrics['rmse']:.4f}")

run_id: 9f9ab13b5b604ae898a9868c4214cd02, run_name: luxuriant-calf-912, val_rmse: 5.1761


In [29]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [30]:
run_id = "9f9ab13b5b604ae898a9868c4214cd02"

model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="my-nyc-xgb-model")

Successfully registered model 'my-nyc-xgb-model'.
Created version '1' of model 'my-nyc-xgb-model'.


<ModelVersion: aliases=[], creation_timestamp=1747802519857, current_stage='None', description=None, last_updated_timestamp=1747802519857, name='my-nyc-xgb-model', run_id='9f9ab13b5b604ae898a9868c4214cd02', run_link=None, source=("file:c:/Sutiwas's Documents/GitHub "
 'Projects/mlops-journey/02-experiment-tracking/mlruns/1/9f9ab13b5b604ae898a9868c4214cd02/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=1>

In [32]:
model_name = "my-nyc-xgb-model"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


C:\Users\UsEr\AppData\Local\Temp\ipykernel_20132\3048196165.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [ ]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

C:\Users\UsEr\AppData\Local\Temp\ipykernel_20132\3681565072.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


MlflowException: Parameter 'version' must be an integer, got 'my-nyc-xgb-model'.

In [36]:
from datetime import datetime

date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}."
)

<ModelVersion: aliases=[], creation_timestamp=1747802519857, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2025-05-21 11:49:26.', last_updated_timestamp=1747802966917, name='my-nyc-xgb-model', run_id='9f9ab13b5b604ae898a9868c4214cd02', run_link=None, source=("file:c:/Sutiwas's Documents/GitHub "
 'Projects/mlops-journey/02-experiment-tracking/mlruns/1/9f9ab13b5b604ae898a9868c4214cd02/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=1>

In [37]:
%time datetime.now().strftime("%Y-%m-%d %H:%M:%S")

CPU times: total: 0 ns
Wall time: 0 ns


'2025-05-21 11:51:31'